In [2]:
import numpy as np  #Importa libreria numerica 
import sympy as sym #simbolica
sym.init_printing() #activa a jupyter para mostrar simbolicamente el output

# Ganancias

In [100]:
v1, v2, Vo          = sym.symbols('v_1,v_2,Vo')
Rin,Rf,R        = sym.symbols('R_in,R_f,R')
vo              = sym.Function('v_o')(v1,v2)
sym.pprint(vo)
#ganancias:
eq1             = sym.Eq(vo.subs(v2,0),-v1*(Rf/R))
sym.pprint(eq1)
eq2             = sym.Eq(vo.subs(v1,0),-v2*(Rf/R))
sym.pprint(eq2)
eq3             = sym.Eq(vo,eq1.rhs+eq2.rhs)
sym.pprint(sym.simplify(eq3))

vₒ(v₁, v₂)
            -R_f⋅v₁ 
vₒ(v₁, 0) = ────────
               R    
            -R_f⋅v₂ 
vₒ(0, v₂) = ────────
               R    
             -R_f⋅(v₁ + v₂) 
vₒ(v₁, v₂) = ───────────────
                    R       


# Calculo de resistencias

## Rin = 50 ohm

In [26]:
eq_Rin          = sym.Eq(Rin,50)
sym.pprint(eq_Rin)
eq_R            = sym.Eq(R,10*Rin)
sym.pprint(eq_R)
sym.pprint(sym.Eq(R,eq_R.rhs.subs(Rin,eq_Rin.rhs)))
eq_Rf           = sym.Eq(Rf,30*eq_R.rhs)
sym.pprint(eq_Rf)
sym.pprint(sym.Eq(Rf,30*eq_R.rhs.subs(Rin,eq_Rin.rhs)))

Rᵢₙ = 50
R = 10⋅Rᵢₙ
R = 500
R_f = 300⋅Rᵢₙ
R_f = 15000


## Rin = 100k ohm

In [91]:
eq_Rin          = sym.Eq(Rin,100e3)
sym.pprint(eq_Rin)
eq_R            = sym.Eq(R,10*Rin)
sym.pprint(eq_R)
eq_Rf           = sym.Eq(Rf,30*eq_R.rhs)
sym.pprint(eq_Rf)
print('Reemplazando: ')
eq_R            = sym.Eq(R,eq_R.rhs.subs(Rin,eq_Rin.rhs)) 
sym.pprint(eq_R)
sym.pprint("{:.1e}".format(eq_R.rhs))
eq_Rf           = sym.Eq(Rf,30*eq_R.rhs.subs(Rin,eq_Rin.rhs))
sym.pprint(eq_Rf)
sym.pprint("{:.1e}".format(eq_Rf.rhs))


Rᵢₙ = 100000.0
R = 10⋅Rᵢₙ
R_f = 300⋅Rᵢₙ
Reemplazando: 
R = 1000000.0
1.0e+6
R_f = 30000000.0
3.0e+7


# Caso de Rf > 1Meg[ohm]
## Se emplea una red T para solucionar este problema
![image](red_T.png)

In [73]:
voTh,iF,vx = sym.symbols('vo_th,i_f,v_x')
Ra,Rb,Rc,P_RaRc    = sym.symbols('Ra,Rb,Rc,P_Ra_Rc')
eq4         = sym.Eq(iF,vx/Ra)
sym.pprint(eq4)
eq_Pall_Ra_Rc = sym.Eq(P_RaRc,1/((1/Ra)+(1/Rc)))
eq_Pall_Ra_Rc = sym.simplify(eq_Pall_Ra_Rc) 
sym.pprint(eq_Pall_Ra_Rc)
eq5         = sym.Eq(vx,voTh*(eq_Pall_Ra_Rc.rhs/(eq_Pall_Ra_Rc.rhs+Rb)))
sym.pprint(eq5)
eq6         = sym.Eq(eq4.lhs,eq4.rhs.subs(vx,eq5.rhs))
sym.pprint(eq6)
eq6         = sym.separatevars(sym.simplify(eq6),force=True) 
sym.pprint(eq6)
print('Con: ')
eq7         = sym.Eq(Rf,voTh/iF)
sym.pprint(eq7)
print('Resulta: ')
eq_Rf_T     = sym.Eq(Rf,(1/eq6.rhs)*voTh)
sym.pprint(eq_Rf_T)

      vₓ
i_f = ──
      Ra
           Ra⋅Rc 
P_Ra_Rc = ───────
          Ra + Rc
            Ra⋅Rc⋅voₜₕ       
vₓ = ────────────────────────
               ⎛ Ra⋅Rc      ⎞
     (Ra + Rc)⋅⎜─────── + Rb⎟
               ⎝Ra + Rc     ⎠
              Rc⋅voₜₕ         
i_f = ────────────────────────
                ⎛ Ra⋅Rc      ⎞
      (Ra + Rc)⋅⎜─────── + Rb⎟
                ⎝Ra + Rc     ⎠
             Rc⋅voₜₕ       
i_f = ─────────────────────
      Ra⋅Rb + Ra⋅Rc + Rb⋅Rc
Con: 
      voₜₕ
R_f = ────
      i_f 
Resulta: 
      Ra⋅Rb + Ra⋅Rc + Rb⋅Rc
R_f = ─────────────────────
                Rc         


## Calculos de Ra,Rb,Rc

In [90]:
eq_Rc       = sym.Eq(Rc,sym.solve(eq_Rf_T,Rc)[0])#el [0] es por que solve devuelve una lista
sym.pprint(eq_Rc)
#Reemplazando y estableciendo Ra=150k;Rb=200k
eq_Rc_val   = sym.Eq(Rc,eq_Rc.rhs.subs({Ra:150e3,Rb:200e3,Rf:eq_Rf.rhs}))
sym.pprint(eq_Rc_val)

         Ra⋅Rb    
Rc = ─────────────
     R_f - Ra - Rb
R_f = 30000000.0
Rc = 1011.80438448567


# Errores en DC
![image](circ_por_black.png)

## Caso de Rin = 50 ohm
### Ganancias

In [123]:
Af,Av,T,Ad  = sym.symbols('A_f,A_v,T,Ad')
#Ganancia de a lazo abierto
#Av = Vo/(V1+V2)
# aplico superposicion
# Vo/V1 = v-/V1 * Vo/v- con V2=0
eq8         = sym.Eq(Vo/v1,(((R**-1+Rf**-1)**-1)/(((R**-1+Rf**-1)**-1)+R))*-Ad*v1)
sym.pprint(eq8)
eq9         = sym.Eq(Vo/v2,(((R**-1+Rf**-1)**-1)/(((R**-1+Rf**-1)**-1)+R))*-Ad*v2)
sym.pprint(eq9)
eq10        = sym.Eq(Vo,(eq9.rhs+eq8.rhs))
sym.pprint(sym.simplify(eq10))
eq_Av       = sym.Eq(Av,(-Ad*Rf)/(R+(2*Rf)))
sym.pprint(eq_Av)



Vo           -Ad⋅v₁         
── = ───────────────────────
v₁   ⎛       1   ⎞ ⎛ 1    1⎞
     ⎜R + ───────⎟⋅⎜─── + ─⎟
     ⎜     1    1⎟ ⎝R_f   R⎠
     ⎜    ─── + ─⎟          
     ⎝    R_f   R⎠          
Vo           -Ad⋅v₂         
── = ───────────────────────
v₂   ⎛       1   ⎞ ⎛ 1    1⎞
     ⎜R + ───────⎟⋅⎜─── + ─⎟
     ⎜     1    1⎟ ⎝R_f   R⎠
     ⎜    ─── + ─⎟          
     ⎝    R_f   R⎠          
     -Ad⋅R_f⋅(v₁ + v₂) 
Vo = ──────────────────
         R + 2⋅R_f     
      -Ad⋅R_f 
Aᵥ = ─────────
     R + 2⋅R_f


In [122]:
#Ganancia de lazo 
#T = Vo/V' = Vo/V- * V-/V'

eq_T        = sym.Eq(T,(((R**-1+R**-1)**-1)/(((R**-1+R**-1)**-1)+Rf))*-Ad)
sym.pprint(eq_T)
eq_T        = sym.simplify(eq_T)
sym.pprint(eq_T)


       -Ad⋅R   
T = ───────────
      ⎛R      ⎞
    2⋅⎜─ + R_f⎟
      ⎝2      ⎠
      -Ad⋅R  
T = ─────────
    R + 2⋅R_f


In [129]:
#Ganancia de lazo cerrado
eq_Af       = sym.Eq(Af,eq_Av.rhs/(1-eq_T.rhs))
sym.pprint(eq_Af)
eq_Af       = sym.simplify(eq_Af)
sym.pprint(eq_Af)
#Reemplazando
sym.pprint(eq_Af.subs({R:500,Rf:15e3,Ad:200e3}))

                -Ad⋅R_f          
A_f = ───────────────────────────
                  ⎛   Ad⋅R      ⎞
      (R + 2⋅R_f)⋅⎜───────── + 1⎟
                  ⎝R + 2⋅R_f    ⎠
          -Ad⋅R_f     
A_f = ────────────────
      Ad⋅R + R + 2⋅R_f
A_f = -29.9908527898991


### Error por Vos
![image](error_vos.png)

In [135]:
vos,dvo_vos         = sym.symbols('V_os,\Delta_Vo')
#Ganancia a lazo abierto en este caso es Vo/Vos = Ad
eq_error_vos    = sym.Eq(dvo_vos,(Ad*vos)/(1-eq_T.rhs))
sym.pprint(eq_error_vos)
sym.pprint(sym.simplify(eq_error_vos))

                Ad⋅Vₒₛ   
\Delta_Vo = ─────────────
               Ad⋅R      
            ───────── + 1
            R + 2⋅R_f    
            Ad⋅Vₒₛ⋅(R + 2⋅R_f)
\Delta_Vo = ──────────────────
             Ad⋅R + R + 2⋅R_f 


### Error por corriente Ios
![image](error_ios.png)

In [145]:
Ipol_n,dvo_Ios         = sym.symbols('I_pol-,\Delta_Vo')
#como en la entrada no inversora no hay corriente entonces no produce caida de tension, por lo tanto no hay salida
#pero aumenta el error total
eq_error_Ios        = sym.Eq(dvo_Ios,(Ipol_n*-Ad*-((R**-1+R**-1+Rf**-1)**-1))/(1-eq_T.rhs))
sym.pprint(eq_error_Ios)
sym.pprint(sym.simplify(eq_error_Ios))

                     Ad⋅Iₚₒₗ₋        
\Delta_Vo = ─────────────────────────
            ⎛ 1    2⎞ ⎛   Ad⋅R      ⎞
            ⎜─── + ─⎟⋅⎜───────── + 1⎟
            ⎝R_f   R⎠ ⎝R + 2⋅R_f    ⎠
             Ad⋅Iₚₒₗ₋⋅R⋅R_f 
\Delta_Vo = ────────────────
            Ad⋅R + R + 2⋅R_f


### Error por CMRR y Ad no infinito



In [151]:

FS, CMRR,dv_Ad_CMRR = sym.symbols('FS, CMRR,\delta_Vo')
eq_error_Ad_CMRR    = sym.Eq(dv_Ad_CMRR,(FS/abs(T)+FS/CMRR))
sym.pprint(sym.sympify(eq_error_Ad_CMRR))


             FS    FS 
\delta_Vo = ─── + ────
            │T│   CMRR


# Errores en Ac
## Ancho de banda plena potencia

In [157]:
SR,Vp,Whp,fhp,t,W = sym.symbols('SR,V_pp,W_hp,f_hp,t,W')
# por definicion el SR = dv(t)/t
SR = sym.Function('SR')()
V_o = sym.Function('V_o')(t)
eq_SR=sym.Eq(SR,sym.diff(V_o))
sym.pprint(eq_SR)

eq_Vo_t=sym.Eq(Vo,Vp*sym.sin(W*t))
sym.pprint(eq_Vo_t)
eq_SR=sym.Eq(SR,sym.diff(eq_Vo_t.rhs,t).subs(t,0))
sym.pprint(eq_SR)
eq_Whp=sym.Eq(Whp,sym.solve(eq_SR,W)[0])
sym.pprint(eq_Whp)

# eq_Whp_val=eq_Whp.subs({SR:500e3, Vp:10})
# sym.pprint(eq_Whp_val)
# eq_fhp_val=sym.Eq(fhp,eq_Whp_val.rhs/(2*np.pi))
# sym.pprint(eq_fhp_val)

       d        
SR() = ──(Vₒ(t))
       dt       
Vo = Vₚₚ⋅sin(W⋅t)
SR() = Vₚₚ⋅W
      SR()
Wₕₚ = ────
      Vₚₚ 
Wₕₚ = 50000.0
fₕₚ = 7957.74715459477
